In [1]:
import pandas as pd
import numpy as np

from google.colab import drive

# Bibliotecas de pré-processamento
from sklearn.preprocessing import LabelEncoder

# Bibliotecas de plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, PassiveAggressiveClassifier, SGDClassifier
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data/merged_df.csv')

In [4]:
from sklearn.preprocessing import StandardScaler

df = df.drop(['record_id', 'idade_faixa'], axis=1)

X_afetam_eficacia = df[['idade_diagnostico','primeiro_IMC','ultima_informacao_paciente', 'tempo_seguimento', 'recidiva_distancia', 'recidiva_regional', 'recidiva_local']]
X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])

<ipython-input-4-6de22f19c6fc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_afetam_eficacia[['tempo_seguimento']] = StandardScaler().fit_transform(X_afetam_eficacia[['tempo_seguimento']])


In [5]:
km = KMeans( n_clusters = 2, init = 'random', max_iter = 300, n_init = 100, random_state = 52 )
km.fit( X_afetam_eficacia )
y_km = km.predict( X_afetam_eficacia )
df['eficacia_tratamento'] = y_km

In [6]:
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico > 80 and ultima_informacao_paciente == 3 and eficacia_tratamento == 1")))

# Este é um sistema de classificação de tratamento em que "0" significa que o tratamento foi bom e "1" significa 
# que foi ruim. Há dois casos apresentados: no primeiro, uma pessoa idosa de 80 anos sobreviveu ao tratamento, indicando 
# que ele foi bom. No segundo caso, não houve sobreviventes entre as pessoas mais velhas e, portanto, o tratamento foi classificado 
# como negativo para essa faixa etária. É importante lembrar que a eficácia do tratamento pode variar de acordo com vários fatores e 
# que cada caso é único.

0
38


In [7]:
df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")

print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 0")))
print(len(df.query("idade_diagnostico < 30 and ultima_informacao_paciente == 0 and eficacia_tratamento == 1")))

# Este modelo classifica o tratamento como bom (0) ou ruim (1). Ele foi testado em dois casos de mulheres com menos
# de 30 anos: uma morreu e a outra sobreviveu. O modelo foi capaz de classificar corretamente cada caso, indicando que 
# tem uma boa capacidade de classificação.

12
0


In [8]:
df = df.query("tratamento == 0 or tratamento == 1")[:]

In [9]:
import numpy as np

df = df.astype('float64')
df = df[np.isfinite(df).all(1)]

In [10]:
from sklearn.model_selection import train_test_split

X = df.drop(['tratamento'], axis=1)
y = df['tratamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 73)

In [11]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

  7%|▋         | 2/29 [00:00<00:10,  2.67it/s]

In [ ]:
models

In [ ]:
classifiers = [AdaBoostClassifier(), 
               RandomForestClassifier(),
               ExtraTreesClassifier(),
               RidgeClassifierCV(), 
               LinearSVC(),
               LogisticRegression(), 
               SVC(), 
               LGBMClassifier(),
               LinearDiscriminantAnalysis(), 
               RidgeClassifier(), 
               CalibratedClassifierCV(), 
               XGBClassifier(),
               NearestCentroid(), 
               SGDClassifier(), 
               BaggingClassifier(),
               BernoulliNB(),
               PassiveAggressiveClassifier(), 
               Perceptron(),
               NuSVC(), 
               DecisionTreeClassifier(), 
               KNeighborsClassifier(),
               LabelSpreading(),
               LabelPropagation(), 
               ExtraTreeClassifier(),
               GaussianNB(),
               DummyClassifier(),
               QuadraticDiscriminantAnalysis()]

precisions = []
recalls = []

for classifier in classifiers:
    clf = classifier.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))

In [ ]:
models['Precision'] = 0
models['Recall'] = 0   

for i in range(0, len(precisions)):
  models['Precision'].iloc[i] = precisions[i]
  models['Recall'].iloc[i] = recalls[i]

models

In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

##GridSearch

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

model = AdaBoostClassifier()

params = {
          'estimator':[None, DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)],
          'n_estimators': [50, 150, 100, 200, 300, 400, 500],
          'learning_rate': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],
          'algorithm': ['SAMME', 'SAMME.R'],
           'random_state': [73]
          }

grid_searchAda = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchAda.fit(X_train, y_train)

print(grid_searchAda.best_params_)
print(grid_searchAda.best_score_)
print(grid_searchAda.score(X_test, y_test))

# {'algorithm': 'SAMME', 'estimator': DecisionTreeClassifier(max_depth=2), 'learning_rate': 0.1, 'n_estimators': 500, 'random_state': 73}
# 0.6920594714828493
# 0.6695371367061357

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier()

params = {
          'n_estimators': [50, 150, 100, 200, 300, 400, 500],
          'max_depth': [5,10,15,20,25],
          'max_features':['sqrt','log2',None],
          'random_state': [73]
          }

grid_searchRF = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchRF.fit(X_train, y_train)

print(grid_searchRF.best_params_)
print(grid_searchRF.best_score_)
print(grid_searchRF.score(X_test, y_test))

# {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 73}
# 0.6911463266674472
# 0.6598493003229279

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

params = {
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150],
    'random_state': [73]
}

grid_searchLGBM = GridSearchCV(estimator=model, param_grid=params, cv=5)

grid_searchLGBM.fit(X_train, y_train)

print(grid_searchLGBM.best_params_)
print(grid_searchLGBM.best_score_)
print(grid_searchLGBM.score(X_test, y_test))

# {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 50, 'random_state': 73}
# 0.6842125988442014
# 0.6727664155005382

##RandomizedSearchCV

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

model = AdaBoostClassifier()

params = {
          'estimator':[None, DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)],
          'n_estimators': [50, 150, 100, 200, 300, 400, 500],
          'learning_rate': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0],
          'algorithm': ['SAMME', 'SAMME.R'],
          'random_state': [73]
          }

randomizedSearchAda = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5, n_iter=50)

randomizedSearchAda.fit(X_train, y_train)

print(randomizedSearchAda.best_params_)
print(randomizedSearchAda.best_score_)
print(randomizedSearchAda.score(X_test, y_test))

# {'random_state': 73, 'n_estimators': 100, 'learning_rate': 0.1, 'estimator': DecisionTreeClassifier(max_depth=2), 'algorithm': 'SAMME'}
# 0.6897510669320249
# 0.6706135629709364

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

random_grid = {
          'n_estimators': [50, 150, 100, 200, 300, 400, 500],
          'max_depth': [5,10,15,20,25],
          'max_features':['sqrt','log2',None],
          'random_state': [73]
               }

randomizedSearchRF = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

randomizedSearchRF.fit(X_train, y_train)

print(randomizedSearchRF.best_params_)
print(randomizedSearchRF.best_score_)
print(randomizedSearchRF.score(X_test, y_test))

# {'random_state': 73, 'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 10}
# 0.6809787626962143
# 0.6641550053821313

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

model = LGBMClassifier()

params = {
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150],
    'random_state': [73]
}

randomizedSearchLGBM = RandomizedSearchCV(estimator=model, param_distributions=params, cv=5)

randomizedSearchLGBM.fit(X_train, y_train)

print(randomizedSearchLGBM.best_params_)
print(randomizedSearchLGBM.best_score_)
print(randomizedSearchLGBM.score(X_test, y_test))

# {'random_state': 73, 'n_estimators': 50, 'max_depth': 10, 'learning_rate': 0.05}
# 0.6842125988442014
# 0.6727664155005382